In [1]:
import pandas as pd

import numpy as np

import re

In [2]:
config_prefix = ['TR', 'BR', 'TI', 'BI']

config_algo_names = ['BinarySearch', 'BubbleSort', 'Factorial', 'Fibonacci', 'IntegerBinary', 'MultiplyMatrix', 'PrimeFactors', 'ReverseString']

config_id_variable = "CASE"

config_answer_variables = [
    ['TR21_01', 'BR21_01', 'TI21_01', 'BI21_01'],
    ['TR22_01', 'BR22_01', 'TI22_01', 'BI22_01'],
    ['TR23_01', 'BR23_01', 'TI23_01', 'BI23_01'],
    ['TR24_01', 'BR24_01', 'TI24_01', 'BI24_01'],
    ['TR25_01', 'BR25_01', 'TI25_01', 'BI25_01'],
    ['TR26_01', 'BR26_01', 'TI26_01', 'BI26_01'],
    ['TR27_01', 'BR27_01', 'TI27_01', 'BI27_01'],
    ['TR28_01', 'BR28_01', 'TI28_01', 'BI28_01']
]

config_time_variables = [
    ['VA01_02','VB01_02','VC01_02','VD01_02'],
    ['VA02_02','VB02_02','VC02_02','VD02_02'],
    ['VA03_02','VB03_02','VC03_02','VD03_02'],
    ['VA04_02','VB04_02','VC04_02','VD04_02'],
    ['VA05_02','VB05_02','VC05_02','VD05_02'],
    ['VA06_02','VB06_02','VC06_02','VD06_02'],
    ['VA07_02','VB07_02','VC07_02','VD07_02'],
    ['VA08_02','VB08_02','VC08_02','VD08_02']
]

config_click_variables = [
    ['VA01_01','VB01_01','VC01_01','VD01_01'],
    ['VA02_01','VB02_01','VC02_01','VD02_01'],
    ['VA03_01','VB03_01','VC03_01','VD03_01'],
    ['VA04_01','VB04_01','VC04_01','VD04_01'],
    ['VA05_01','VB05_01','VC05_01','VD05_01'],
    ['VA06_01','VB06_01','VC06_01','VD06_01'],
    ['VA07_01','VB07_01','VC07_01','VD07_01'],
    ['VA08_01','VB08_01','VC08_01','VD08_01']
]


config_answer_patterns = [
    ['.*3.*', '.*3.*', '.*3.*', '.*3.*'],
    ['.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*','.*3.*16.*23.*42.*61.*75.*536.*'],
    ['.*6.*','.*6.*','.*120.*','.*120.*'],
    ['.*2.*', '.*2.*','.*2.*','.*2.*'],
    ['.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*','.*1.*0.*0.*0.*1.*'],
    ['.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*','.*6.*6.*6.*12.*12.*12.*18.*18.*18.*'],
    ['.*3.*5.*','.*3.*5.*','.*3.*5.*','.*3.*5.*'],
    ['.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*','.*gnikcar[tT].*ey[eE].*']
]


config_datasheet = r'./BookGDCopy.xlsx'

In [3]:
df_matrix = []
raw = pd.read_excel(config_datasheet)

# load data
for row, _stuff in enumerate(config_algo_names):
    df_array = []
    for col, _stuff in enumerate(config_prefix):
        df = pd.DataFrame(raw, columns = [config_id_variable, 
                                          config_answer_variables[row][col], 
                                          config_time_variables[row][col],
                                          config_click_variables[row][col],])
        df = df.iloc[1:]
        df = df.dropna()
        df_array.append(df)
        
        
    df_matrix.append(df_array)

In [4]:
cols = ["id", "response_time", "correctness", "flag", "algo_name", "click_data"]
df_BU_R = pd.DataFrame(columns=cols)
df_BU_I = pd.DataFrame(columns=cols)
df_TD_R = pd.DataFrame(columns=cols)
df_TD_I = pd.DataFrame(columns=cols)

for row_idx, df_row in enumerate(df_matrix):
    algo_name = config_algo_names[row_idx]
    for col, df in enumerate(df_row): 
        pattern = config_answer_patterns[row_idx][col]
        regex = re.compile(pattern)
        
        for index, row in df.iterrows():

            id_value =  row["CASE"]
            response_time = int(row[config_time_variables[row_idx][col]])
            answer = row[config_answer_variables[row_idx][col]]
            click_data = row[config_click_variables[row_idx][col]]
            result = regex.match(str(answer))
            correctness = False
            
            if result is not None:
                correctness = True
                
            flag = ""
            data = pd.DataFrame([[id_value, response_time, correctness, flag, algo_name, click_data]], columns=cols)
                  
            if col == 0:
                df_BU_R = df_BU_R.append(data)
            elif col == 1:
                df_BU_I = df_BU_I.append(data)
            elif col == 2:
                df_TD_R = df_TD_R.append(data)
            else:
                df_TD_I = df_TD_I.append(data)

In [5]:
df_array_independent = [df_BU_R, df_BU_I, df_TD_R, df_TD_I]

In [6]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
cols = ["Subject","ProgrammingStyle", "Comprehension", "ResponseTime", "Algorithm", "Flag", "Correctness", "ClickData"]

def fill(list_data, dataframe, coding_style, comprehension_style):
    for idx, row in dataframe.iterrows():
        response_time = row["response_time"]
        algorithm = row["algo_name"]
        flag = row["flag"]
        click_data = row["click_data"]
        correctness = row["correctness"]
        id = row["id"]
        list_data.append([id,coding_style, comprehension_style, response_time, algorithm, flag, correctness, click_data])

data = []
fill(data, df_BU_R, "R", "BU")
fill(data, df_BU_I, "I", "BU")
fill(data, df_TD_R, "R", "TD")
fill(data, df_TD_I, "I", "TD")
df = pd.DataFrame(data, columns=cols)
print (df)
df.to_excel("./results/preprocessed.xlsx", index=False)

     Subject ProgrammingStyle Comprehension  ResponseTime      Algorithm Flag  \
0        125                R            BU         18060   BinarySearch        
1        131                R            BU          8026   BinarySearch        
2        154                R            BU         18033   BinarySearch        
3        172                R            BU         15172   BinarySearch        
4        194                R            BU         18028   BinarySearch        
..       ...              ...           ...           ...            ...  ...   
711      354                I            TD          3088  ReverseString        
712      357                I            TD         21061  ReverseString        
713      372                I            TD         21060  ReverseString        
714      379                I            TD          6213  ReverseString        
715      392                I            TD          3118  ReverseString        

     Correctness           